In [1]:
import gc
import torch
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import transformers
import os

gc.collect()
torch.cuda.empty_cache()

metric_collector = []


# List of glue tasks
#GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2",  "wnli"]
#GLUE_TASKS = ["qqp"]

In [2]:

for task in GLUE_TASKS:
    
    #List of glue keys
    task_to_keys = {
        "cola": ("sentence", None),
        "mnli": ("premise", "hypothesis"),
        "mnli-mm": ("premise", "hypothesis"),
        "mrpc": ("sentence1", "sentence2"),
        "qnli": ("question", "sentence"),
        "qqp": ("question1", "question2"),
        "rte": ("sentence1", "sentence2"),
        "sst2": ("sentence", None),
        "stsb": ("sentence1", "sentence2"),
        "wnli": ("sentence1", "sentence2"),
    }
    
    #Select task
    #task = "rte"  #cola, mrpc
    batch_size = 10 #10 normally, 8 for qnli
    
    # Load dataset based on task variable
    actual_task = "mnli" if task == "mnli-mm" else task
    dataset = load_dataset("glue", actual_task)
    metric = load_metric('glue', actual_task)
    
    #Collect sentence keys and labels
    sentence1_key, sentence2_key = task_to_keys[task]
    
    # Number of logits to output
    num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
    
    

    ###############################################
    
    #         DEBERTA SECTION
    
    ###############################################
    
    
    ###  Tokenizing Section  ####
    
    #Load model
    model_checkpoint = "microsoft/deberta-v3-small"
    
    # Create tokenizer for respective model
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)
    
    def tokenizer_func(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True,)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)
    
    # tokenize sentence(s)
    encoded_dataset = dataset.map(tokenizer_func, batched=True)
    
    #model_checkpoint = "deberta-v3-small_baseline_cola/"
    model_checkpoint = "deberta-v3-small_baseline_"+actual_task+"/"
    
    ###  Model Section  ####
    
    # Create model and attach ForSequenceClassification head
    model_deberta = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    
    # Type of metric for given task
    metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
    
    args = TrainingArguments(
        f"{model_checkpoint}-finetuned-Testing-{task}",
        evaluation_strategy = "epoch",
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        metric_for_best_model=metric_name,
        eval_accumulation_steps=5
    )
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
    trainer = Trainer(
        model_deberta,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.evaluate()
    

    
    ### Collect Predictions  ###
    
    prediction_deberta = trainer.predict(encoded_dataset[validation_key])
    

    
    ## Clear the Cache
    gc.collect()
    torch.cuda.empty_cache()
    
    
    ###############################################
    
    #         ELECTRA SECTION
    
    ###############################################
    
    
    ###  Tokenizing Section  ####
    
    #Load model
    model_checkpoint = "google/electra-small-discriminator"
    
    # Create tokenizer for respective model
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)
    
    def tokenizer_func(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True,)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)
    
    # tokenize sentence(s)
    encoded_dataset = dataset.map(tokenizer_func, batched=True)
    
    #model_checkpoint = "electra-small-discriminator-finetuned-cola/"
    #model_checkpoint = "Electra_fintuned_cola/"
    model_checkpoint = "Electra_fintuned_"+actual_task+"/"
    
    ###  Model Section  ####
    
    # Create model and attach ForSequenceClassification head
    model_electra = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    
    # Type of metric for given task
    metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
    
    
    args = TrainingArguments(
        f"{model_checkpoint}-finetuned-Testing-{task}",
        evaluation_strategy = "epoch",
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        metric_for_best_model=metric_name,
        eval_accumulation_steps=5
    )
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
    trainer = Trainer(
        model_electra,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.evaluate()
    
    

    
    ### Collect Predictions  ###
    ## Clear the Cache
    gc.collect()
    torch.cuda.empty_cache()
    prediction_electra = trainer.predict(encoded_dataset[validation_key])
    

    
    
    ## Clear the Cache
    gc.collect()
    torch.cuda.empty_cache()
    


    ###############################################
    
    #         XLNET SECTION
    
    ###############################################
    
    
    ###  Tokenizing Section  ####
    
    #Load model
    model_checkpoint = "xlnet-base-cased"
    
    # Create tokenizer for respective model
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)
    
    def tokenizer_func(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True,)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)
    
    # tokenize sentence(s)
    encoded_dataset = dataset.map(tokenizer_func, batched=True)
    
    #model_checkpoint = "electra-small-discriminator-finetuned-cola/"
    #model_checkpoint = "Electra_fintuned_cola/"
    model_checkpoint = "xlnet-base-cased_baseline_"+actual_task+"/"
    
    ###  Model Section  ####
    
    # Create model and attach ForSequenceClassification head
    model_xlnet = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    
    # Type of metric for given task
    metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
    
    
    args = TrainingArguments(
        f"{model_checkpoint}-finetuned-Testing-{task}",
        evaluation_strategy = "epoch",
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        metric_for_best_model=metric_name,
        eval_accumulation_steps=5
    )
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
    trainer = Trainer(
        model_xlnet,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.evaluate()
    

    
    ### Collect Predictions  ###
    ## Clear the Cache
    gc.collect()
    torch.cuda.empty_cache()
    prediction_xlnet = trainer.predict(encoded_dataset[validation_key])
    

    
    ## Clear the Cache
    gc.collect()
    torch.cuda.empty_cache()
    

    
    

    ###############################################
    
    # Combine Model Predicions to create Input Features
    
    ###############################################
    
    
    import pandas as pd
    
    #Labels
    val_labels = prediction_deberta.label_ids
    
    
    #DeBERTa
    df_deberta = pd.DataFrame(prediction_deberta[0])
    df_deberta=df_deberta.rename(columns=dict(zip(df_deberta.columns,['deberta_'+str(col) for col in df_deberta.columns])))
    print(df_deberta.head(),'\n')
    
    
    #Electra
    df_electra = pd.DataFrame(prediction_electra[0])
    df_electra=df_electra.rename(columns=dict(zip(df_electra.columns,['electra_'+str(col) for col in df_electra.columns])))
    print(df_electra.head(),'\n')
    
    
    #XLNet
    df_xlnet = pd.DataFrame(prediction_xlnet[0])
    df_xlnet=df_xlnet.rename(columns=dict(zip(df_xlnet.columns,['xlnet_'+str(col) for col in df_xlnet.columns])))
    print(df_xlnet.head(),'\n')
    
    

    
    
    #Combine the dataframes
    df_combine = pd.concat([df_deberta, df_electra, df_xlnet], axis=1)
    df_combine.head()
    
    
    ###############################################
    
    #         ENSEMBLE SECTION
    
    ###############################################
    
    
    # Importing the required packages
    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import matthews_corrcoef
    from sklearn.metrics import classification_report
    from sklearn.metrics import roc_auc_score
    

    
    # Split the dataset into train and test
    
    X_train, X_test, y_train, y_test = train_test_split(df_combine, val_labels, test_size=0.3, random_state=100)
    X_train.head()
    
    
    # Perform training with random forest with all columns
    # Initialize random forest classifier
    clf = RandomForestClassifier(n_estimators=100)
    
    # Perform training
    clf.fit(X_train, y_train)
    
    # Predicton on test using all features
    y_pred = clf.predict(X_test)
    y_pred_score = clf.predict_proba(X_test)
    

    
    # Print basic Report, then specify for the model
    
    print("\n")
    print("Results Using All Features: \n")
    
    print("Classification Report: ")
    print(classification_report(y_test,y_pred))
    print("\n")
    
    if metric_name == 'accuracy':
        ensemble_score = accuracy_score(y_test, y_pred)
        try:
            ensemble_f = f1_score(y_test, y_pred)
        except:
            pass
    
    elif metric_name == 'matthews_correlation':
        ensemble_score = matthews_corrcoef(y_test, y_pred)
        ensemble_f = 999
    
    elif metric_name == "pearson":
        ensemble_score = accuracy_score(y_test, y_pred)
        ensemble_f = 999
    else:
        ensemble_score = 999
        ensemble_f = 999
        print('ERROR')
    
    try:
        deberta_f = prediction_deberta.metrics['test_f1'], 
        electra_f = prediction_electra.metrics['test_f1'], 
        xlnet_f = prediction_xlnet.metrics['test_f1']
    except:
        ensemble_f = deberta_f = electra_f = xlnet_f = 999
    
    print("Accuracy : ", ensemble_score * 100, '\nFscore : ', ensemble_f)
        
    print('-------------------')
    print("DeBERTa : ", prediction_deberta.metrics['test_'+metric_name]*100)
    print("Electra : ", prediction_electra.metrics['test_'+metric_name]*100)
    print("XLNet : ", prediction_xlnet.metrics['test_'+metric_name]*100)
    
    
    metric_collector.append([task,
                             ensemble_score,
                             prediction_deberta.metrics['test_'+metric_name], 
                             prediction_electra.metrics['test_'+metric_name], 
                             prediction_xlnet.metrics['test_'+metric_name],
                             ensemble_f, deberta_f, electra_f, xlnet_f])
    

Reusing dataset glue (/home/ubuntu/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 10


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file xlnet-base-cased_baseline_wnli/config.json
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetForSequenceClassification"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "problem_type": "single_label_classification",
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "untie_r": true,
  "

The following columns in the test set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 71
  Batch size = 10


   deberta_0  deberta_1
0  -0.095248  -0.074567
1  -0.095816  -0.077622
2  -0.095825  -0.074736
3  -0.092846  -0.072272
4  -0.093128  -0.073961 

   electra_0  electra_1
0   0.063873   0.019810
1   0.070295   0.057328
2   0.062983   0.051700
3   0.066377   0.015463
4   0.070587   0.046793 

    xlnet_0   xlnet_1
0  0.353091 -0.020062
1  0.336589 -0.030646
2  0.345355 -0.035084
3  0.356083 -0.065204
4  0.369883 -0.018039 



Results Using All Features: 

Classification Report: 
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        12
           1       0.70      0.70      0.70        10

    accuracy                           0.73        22
   macro avg       0.72      0.72      0.72        22
weighted avg       0.73      0.73      0.73        22



Accuracy :  72.72727272727273 
Fscore :  999
-------------------
DeBERTa :  43.66197183098591
Electra :  46.478873239436616
XLNet :  56.33802816901409


In [3]:
print('Done')

Done


In [8]:
ensemble_metrics = pd.DataFrame(metric_collector, columns = ['Task','Ensemble', 'DeBERTa', 'Electra', 'XLNet', 
                                                             'Ensemble_f','DeBERTa_f', 'Electra_f', 'XLNet_f', ])
ensemble_metrics.head(15)

,Task,Ensemble,DeBERTa,Electra,XLNet,Ensemble_f,DeBERTa_f,Electra_f,XLNet_f
0,cola,0.658650,0.620829,0.607029,0.399916,999.000000,999,999,999.000000
1,mnli,0.875042,0.874580,0.817422,0.856648,999.000000,999,999,999.000000
2,mnli-mm,0.871186,0.872356,0.821298,0.857811,999.000000,999,999,999.000000
3,mrpc,0.869919,0.865196,0.882353,0.892157,0.905882,"(0.9029982363315697,)","(0.9145907473309609,)",0.922535
4,qnli,0.917023,0.915431,0.889072,0.877906,999.000000,999,999,999.000000
5,qqp,0.907082,0.906307,0.900322,0.873782,0.875180,"(0.8749917497194905,)","(0.8659972068896721,)",0.833317
6,rte,0.702381,0.667870,0.682310,0.740072,999.000000,999,999,999.000000
7,sst2,0.958015,0.934633,0.917431,0.940367,999.000000,999,999,999.000000
8,wnli,0.727273,0.436620,0.464789,0.563380,999.000000,999,999,999.000000


In [31]:
#Remove tuples from the f-scores in some columns

for column in ['DeBERTa_f', 'Electra_f']:
    col = ensemble_metrics[column]

    for val in range(col.shape[0]):
        #Correct tuples
        if type(col[val]) == tuple:
            ensemble_metrics[column][val] = col[val][0]
            
ensemble_metrics.head(15)

,Task,Ensemble,DeBERTa,Electra,XLNet,Ensemble_f,DeBERTa_f,Electra_f,XLNet_f
0,cola,0.658650,0.620829,0.607029,0.399916,999.000000,999,999,999.000000
1,mnli,0.875042,0.874580,0.817422,0.856648,999.000000,999,999,999.000000
2,mnli-mm,0.871186,0.872356,0.821298,0.857811,999.000000,999,999,999.000000
3,mrpc,0.869919,0.865196,0.882353,0.892157,0.905882,0.902998,0.914591,0.922535
4,qnli,0.917023,0.915431,0.889072,0.877906,999.000000,999,999,999.000000
5,qqp,0.907082,0.906307,0.900322,0.873782,0.875180,0.874992,0.865997,0.833317
6,rte,0.702381,0.667870,0.682310,0.740072,999.000000,999,999,999.000000
7,sst2,0.958015,0.934633,0.917431,0.940367,999.000000,999,999,999.000000
8,wnli,0.727273,0.436620,0.464789,0.563380,999.000000,999,999,999.000000


In [32]:
ensemble_metrics.to_csv('Ensemble_save.csv')
ensemble_metrics.to_excel('Ensemble_save.xlsx')